## MSCA 32018 1 Natural Language Processing and Cognitive Computing

### Assignment 4

**Author: Targoon Siripanichpong**

Create classification model, predicting the outcome of food safety inspection based on the inspectors’ comments

- Leverage the results of your homework from Week-1 and Week-2 to extract free-form text comments from inspectors
- Discard the text from “Health Code” – only keep inspectors’ comments
- Build classification model, predicting the outcome of inspection – your target variable is “Results”
- Explain why you selected a particular text pre-processing technique
- Visualize results of at least two text classifiers and select the most robust one
- You can choose to build a binary classifier (limiting your data to Pass / Fail) or multinomial classifier with all available values in Results


#### Exploratory Analysis

**Import Data & Data Cleaning**

In [1]:
import pandas as pd
import re
food_inspection = pd.read_csv('../Data/Food_Inspections.csv')

In [2]:
# Drop NaNs and reset index
food_inspection.dropna(subset=['Violations'], how='all', inplace = True)
food_inspection.reset_index(inplace=True)

In [3]:
len(food_inspection)

135592

In [4]:
food_inspection.head()

,index,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,0,2282603,A. J. HUDSON'S PUBLIC HOUSE,A. J. HUDSON'S PUBLIC HOUSE,1248488.0,Restaurant,Risk 1 (High),3801-3805 N ASHLAND AVE,CHICAGO,IL,60613.0,04/04/2019,Complaint Re-Inspection,Pass w/ Conditions,25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCO...,41.950735,-87.668813,"(41.950735168842144, -87.66881290118066)"
1,1,2282582,CHILDREN'S LEARNING PLACE,CHILDREN'S LEARNING PLACE,2369232.0,Children's Services Facility,Risk 1 (High),3532 N HALSTED ST,CHICAGO,IL,60657.0,04/04/2019,License,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.946415,-87.649605,"(41.94641497610327, -87.64960508860683)"
2,2,2282567,NEW YORK HALAL PIZZA,NEW YORK HALAL PIZZA,2506788.0,Restaurant,Risk 1 (High),2262 W DEVON AVE,CHICAGO,IL,60659.0,04/04/2019,Canvass,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.997883,-87.687389,"(41.99788329809707, -87.68738928391146)"
3,4,2282535,PHILLYS ORIGINAL,PHILLY'S ORIGINAL,2304565.0,Restaurant,Risk 2 (Medium),8245 S KEDZIE AVE,CHICAGO,IL,60652.0,04/03/2019,Canvass Re-Inspection,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.742985,-87.702064,"(41.74298525218482, -87.70206428481328)"
4,5,2282536,"HOST INTERNATIONAL INC, CHILIS T-2",CHILI'S TOO (T2 F4),34192.0,Restaurant,Risk 1 (High),11601 W TOUHY AVE,CHICAGO,IL,60666.0,04/03/2019,Canvass,Pass w/ Conditions,16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED...,42.008536,-87.914428,"(42.008536400868735, -87.91442843927047)"


In [5]:
print(food_inspection['Violations'][51])

3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments: OBSERVED NO WRITTEN EMPLOYEE HEALTH POLICY ON THE PREMISES.  MANAGEMENT INSTRUCTED TO PROVIDE A SIGNED EMPLOYEE HEALTH POLICY FOR EACH EMPLOYEE.  PRIORITY FOUNDATION 7-38-010.  NO CITATION ISSUED. | 5. PROCEDURES FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS - Comments: OBSERVED NO CLEAN UP PROCEDURE AND SUPPLIES ON THE PREMISES FOR VOMIT AND DIARRHEAL EVENTS.  MANAGEMENT INSTRUCTED TO PROVIDE BOTH.  PRIORITY FOUNDATION 7-38-005.  NO CITATION ISSUED. | 10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE - Comments: OBSERVED NO HAND SOAP AT THE EXPOSED HANDWASHING SINK IN THE FRONT FOOD PREP AREA WHERE COFFEE IS SERVED.  MANAGEMENT PROVIDED HAND SOAP AT THE SINK AT MY REQUEST.  PRIORITY FOUNDATION 7-38-030(C).  CITATION ISSUED. | 38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - Comments: OBSERVED LIVE INSECTS AND EVIDENCE OF RODENT ACTIVITY IN THE REAR FOOD PREP AREA A

**Split violations to list**

In [6]:
#Split by "|"
split_term = ' \| '

food_inspection['Violation_List'] = [re.split(split_term, violations) for violations in food_inspection['Violations']]

In [7]:
len(food_inspection['Violation_List'])

135592

**Extract only Comment Section**

In [8]:
#Only extract the Comment Section
split_term = ' - Comments: '
food_inspection['Violation_List_Comments'] = [[re.split(split_term, violation) for violation in violation_list] for violation_list in food_inspection['Violation_List']]

In [9]:
food_inspection['Violation_List_Comments'][0]

[['25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCOOKED FOOD',
  'ALL MENU ITEMS THAT PERTAIN TO THE CONSUMER ADVISORY MUST BE PROPERLY REMINDED AND DISCLOSED ON MENUS. PRIORITY FOUNDATION VIOLATION. NO CITATION ISSUED.'],
 ['40. PERSONAL CLEANLINESS',
  ' 2-402.11 ALL FOOD PREPARERS WITH BEARDS, MUST HAVE BEARD/HAIR RESTRAINTS'],
 ['55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN',
  ' 6-201.17 EXPANDING SPRAY FOAM SEALANT USED ON THE WALL BEHIND THE BAR TO SEAL AROUND ELECTRICAL AND PLUMBING OPENINGS. MUST REMOVE TO MAKE SMOOTH AND CLEANABLE.']]

In [10]:
food_inspection['Violation_List_Comments_Only'] = [[violation[1] for violation in violation_list if len(violation) == 2] for violation_list in food_inspection['Violation_List_Comments']] 

In [11]:
food_inspection['Violation_List_Comments_Only'][51]

['OBSERVED NO WRITTEN EMPLOYEE HEALTH POLICY ON THE PREMISES.  MANAGEMENT INSTRUCTED TO PROVIDE A SIGNED EMPLOYEE HEALTH POLICY FOR EACH EMPLOYEE.  PRIORITY FOUNDATION 7-38-010.  NO CITATION ISSUED.',
 'OBSERVED NO CLEAN UP PROCEDURE AND SUPPLIES ON THE PREMISES FOR VOMIT AND DIARRHEAL EVENTS.  MANAGEMENT INSTRUCTED TO PROVIDE BOTH.  PRIORITY FOUNDATION 7-38-005.  NO CITATION ISSUED.',
 'OBSERVED NO HAND SOAP AT THE EXPOSED HANDWASHING SINK IN THE FRONT FOOD PREP AREA WHERE COFFEE IS SERVED.  MANAGEMENT PROVIDED HAND SOAP AT THE SINK AT MY REQUEST.  PRIORITY FOUNDATION 7-38-030(C).  CITATION ISSUED.',
 'OBSERVED LIVE INSECTS AND EVIDENCE OF RODENT ACTIVITY IN THE REAR FOOD PREP AREA AND STORAGE CLOSET.  OBSERVED APPROXIMATELY 40 LIVE ANTS CRAWLING ON THE FLOOR IN THE STORAGE CLOSET AND UNDER THE 1 COMPARTMENT SINK IN THE REAR FOOD PREP AREA.  ALSO, OBSERVED APPROXIMATELY 20 MOUSE DROPPINGS ON THE FLOOR IN THE STORAGE CLOSET UNDER THE MOP SINK AND IN THE CORNER OF THE REAR FOOD PREP ARE

In [12]:
#Flatten the comments
food_inspection['Violation_List_Comments_Only'] = [' '.join(violation_list) for violation_list in food_inspection['Violation_List_Comments_Only']]

In [13]:
food_inspection['Violation_List_Comments_Only'][51]

'OBSERVED NO WRITTEN EMPLOYEE HEALTH POLICY ON THE PREMISES.  MANAGEMENT INSTRUCTED TO PROVIDE A SIGNED EMPLOYEE HEALTH POLICY FOR EACH EMPLOYEE.  PRIORITY FOUNDATION 7-38-010.  NO CITATION ISSUED. OBSERVED NO CLEAN UP PROCEDURE AND SUPPLIES ON THE PREMISES FOR VOMIT AND DIARRHEAL EVENTS.  MANAGEMENT INSTRUCTED TO PROVIDE BOTH.  PRIORITY FOUNDATION 7-38-005.  NO CITATION ISSUED. OBSERVED NO HAND SOAP AT THE EXPOSED HANDWASHING SINK IN THE FRONT FOOD PREP AREA WHERE COFFEE IS SERVED.  MANAGEMENT PROVIDED HAND SOAP AT THE SINK AT MY REQUEST.  PRIORITY FOUNDATION 7-38-030(C).  CITATION ISSUED. OBSERVED LIVE INSECTS AND EVIDENCE OF RODENT ACTIVITY IN THE REAR FOOD PREP AREA AND STORAGE CLOSET.  OBSERVED APPROXIMATELY 40 LIVE ANTS CRAWLING ON THE FLOOR IN THE STORAGE CLOSET AND UNDER THE 1 COMPARTMENT SINK IN THE REAR FOOD PREP AREA.  ALSO, OBSERVED APPROXIMATELY 20 MOUSE DROPPINGS ON THE FLOOR IN THE STORAGE CLOSET UNDER THE MOP SINK AND IN THE CORNER OF THE REAR FOOD PREP AREA BEHIND THE 

#### Text Tokenization & Cleaning

In [14]:
import nltk;
nltk.download('popular', halt_on_error=False);
nltk.download('all', halt_on_error=False);

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package movie_reviews is already

[nltk_data]    |   Package lin_thesaurus is already up-to-date!
[nltk_data]    | Downloading package mac_morpho to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package mac_morpho is already up-to-date!
[nltk_data]    | Downloading package machado to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package machado is already up-to-date!
[nltk_data]    | Downloading package masc_tagged to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package masc_tagged is already up-to-date!
[nltk_data]    | Downloading package moses_sample to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package moses_sample is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to
[nltk_data]    |     /home/targoon/nltk_data...
[nltk_data]    |   Package names is 

True

In [15]:
from nltk.text import Text
import sys
import multiprocessing
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import inflection as inf

In [16]:
pool = multiprocessing.Pool(processes=7)
food_inspection['Violation_List_Comments_Tokenized'] = pool.map(nltk.tokenize.word_tokenize, food_inspection['Violation_List_Comments_Only'])

In [17]:
food_inspection['Violation_List_Comments_Tokenized'][51]

['OBSERVED',
 'NO',
 'WRITTEN',
 'EMPLOYEE',
 'HEALTH',
 'POLICY',
 'ON',
 'THE',
 'PREMISES',
 '.',
 'MANAGEMENT',
 'INSTRUCTED',
 'TO',
 'PROVIDE',
 'A',
 'SIGNED',
 'EMPLOYEE',
 'HEALTH',
 'POLICY',
 'FOR',
 'EACH',
 'EMPLOYEE',
 '.',
 'PRIORITY',
 'FOUNDATION',
 '7-38-010',
 '.',
 'NO',
 'CITATION',
 'ISSUED',
 '.',
 'OBSERVED',
 'NO',
 'CLEAN',
 'UP',
 'PROCEDURE',
 'AND',
 'SUPPLIES',
 'ON',
 'THE',
 'PREMISES',
 'FOR',
 'VOMIT',
 'AND',
 'DIARRHEAL',
 'EVENTS',
 '.',
 'MANAGEMENT',
 'INSTRUCTED',
 'TO',
 'PROVIDE',
 'BOTH',
 '.',
 'PRIORITY',
 'FOUNDATION',
 '7-38-005',
 '.',
 'NO',
 'CITATION',
 'ISSUED',
 '.',
 'OBSERVED',
 'NO',
 'HAND',
 'SOAP',
 'AT',
 'THE',
 'EXPOSED',
 'HANDWASHING',
 'SINK',
 'IN',
 'THE',
 'FRONT',
 'FOOD',
 'PREP',
 'AREA',
 'WHERE',
 'COFFEE',
 'IS',
 'SERVED',
 '.',
 'MANAGEMENT',
 'PROVIDED',
 'HAND',
 'SOAP',
 'AT',
 'THE',
 'SINK',
 'AT',
 'MY',
 'REQUEST',
 '.',
 'PRIORITY',
 'FOUNDATION',
 '7-38-030',
 '(',
 'C',
 ')',
 '.',
 'CITATION',
 'ISSU

In [18]:
#Clean-up unused columns
food_inspection.drop(columns=['Violation_List_Comments','Violation_List_Comments_Only'],inplace=True)

In [19]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def text_preprocessing(words):
    
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wnl = WordNetLemmatizer()
    
    # Remove single-character tokens (mostly punctuation)
    words = [word for word in words if len(word) > 1]

    # Remove numbers
    words = [word for word in words if not word.isnumeric()]

    # Remove punctuation
    words = [word for word in words if word.isalpha()]

    # Lowercase all words (default_stopwords are lowercase too)
    words = [word.lower() for word in words]

    # Remove stopwords
    #words = [word for word in words if word not in stopwords]
    
    # Get PoS Tagging
    words_pos = nltk.pos_tag(words)
    words_pos = [(word, get_wordnet_pos(pos)) for (word, pos) in words_pos]
    
    # Lemmatize with POS tagging
    words_pos = [(wnl.lemmatize(word, pos), pos) for (word, pos) in words_pos]
    
    # Singularize words
    words = [inf.singularize(word) if pos == 'n' else word for (word, pos) in words_pos]
    
    return(words)

In [20]:
pool = multiprocessing.Pool(processes=7)
food_inspection['Violation_List_Comments_Tokenized_Cleaned'] = pool.map(text_preprocessing, food_inspection['Violation_List_Comments_Tokenized'])

In [21]:
food_inspection['Violation_List_Comments_Cleaned'] = [' '.join(token) for token in food_inspection['Violation_List_Comments_Tokenized_Cleaned']]

In [22]:
food_inspection.head()

,index,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,...,Inspection Type,Results,Violations,Latitude,Longitude,Location,Violation_List,Violation_List_Comments_Tokenized,Violation_List_Comments_Tokenized_Cleaned,Violation_List_Comments_Cleaned
0,0,2282603,A. J. HUDSON'S PUBLIC HOUSE,A. J. HUDSON'S PUBLIC HOUSE,1248488.0,Restaurant,Risk 1 (High),3801-3805 N ASHLAND AVE,CHICAGO,IL,...,Complaint Re-Inspection,Pass w/ Conditions,25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCO...,41.950735,-87.668813,"(41.950735168842144, -87.66881290118066)",[25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERC...,"[ALL, MENU, ITEMS, THAT, PERTAIN, TO, THE, CON...","[menu, item, pertain, consumer, advisory, must...",menu item pertain consumer advisory must prope...
1,1,2282582,CHILDREN'S LEARNING PLACE,CHILDREN'S LEARNING PLACE,2369232.0,Children's Services Facility,Risk 1 (High),3532 N HALSTED ST,CHICAGO,IL,...,License,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.946415,-87.649605,"(41.94641497610327, -87.64960508860683)","[1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNO...","[NO, PERSON, IN, CHARGE, PRESENT, AT, THE, TIM...","[person, charge, present, time, inspection, pr...",person charge present time inspection priority...
2,2,2282567,NEW YORK HALAL PIZZA,NEW YORK HALAL PIZZA,2506788.0,Restaurant,Risk 1 (High),2262 W DEVON AVE,CHICAGO,IL,...,Canvass,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.997883,-87.687389,"(41.99788329809707, -87.68738928391146)",[47. FOOD & NON-FOOD CONTACT SURFACES CLEANABL...,"[MUST, CLEAN, AND, SANITIZE, ALL, CHOPPING, BO...","[must, clean, sanitize, chopping, board, maint...",must clean sanitize chopping board maintain fo...
3,4,2282535,PHILLYS ORIGINAL,PHILLY'S ORIGINAL,2304565.0,Restaurant,Risk 2 (Medium),8245 S KEDZIE AVE,CHICAGO,IL,...,Canvass Re-Inspection,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.742985,-87.702064,"(41.74298525218482, -87.70206428481328)",[5. PROCEDURES FOR RESPONDING TO VOMITING AND ...,"[OBSERVED, NO, PROCEDURE, FOR, RESPONDING, TO,...","[observed, procedure, respond, vomit, diarrhea...",observed procedure respond vomit diarrhea even...
4,5,2282536,"HOST INTERNATIONAL INC, CHILIS T-2",CHILI'S TOO (T2 F4),34192.0,Restaurant,Risk 1 (High),11601 W TOUHY AVE,CHICAGO,IL,...,Canvass,Pass w/ Conditions,16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED...,42.008536,-87.914428,"(42.008536400868735, -87.91442843927047)",[16. FOOD-CONTACT SURFACES: CLEANED & SANITIZE...,"[FOUND, LOW, TEMPERATURE, DISH, WASHING, MACHI...","[find, low, temperature, dish, wash, machine, ...",find low temperature dish wash machine final s...


#### Model Preparation

In [24]:
food_inspection["Results"].value_counts()

Pass                  80182
Fail                  32682
Pass w/ Conditions    22238
No Entry                420
Not Ready                43
Out of Business          27
Name: Results, dtype: int64

**Limit the model to Pass, Fail, and Pass with conditions.**

In [25]:
#Drop rows with low volume of entries (No Entry, Not Ready, OOB)
food_inspection.drop(food_inspection[food_inspection['Results'].isin(['No Entry', 'Not Ready', 'Out of Business'])].index, inplace=True)

In [26]:
food_inspection["Results"].value_counts()

Pass                  80182
Fail                  32682
Pass w/ Conditions    22238
Name: Results, dtype: int64

In [30]:
#Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(food_inspection['Violation_List_Comments_Cleaned'], food_inspection["Results"], test_size=0.3, random_state=42)

In [31]:
y_train.value_counts()

Pass                  56097
Fail                  22934
Pass w/ Conditions    15540
Name: Results, dtype: int64

#### Text Vectorization

In [34]:
X_train[94980]

'raw wood door womens wahroom door go hallway must seal paint raw wood must clean detail inside cabinet behind bar inside island cabinet use storage area organize discard unnecessary item use prevent harborage crack floor tile inside woman wahroom behind bar must floor tile must clean detail floor inside front small storage room repair concrete floor make smooth easily clenable fan guard inside door reach cooler vent cover washroom heavy dust accumulate must clean detail maintain self close device woman washroom door must install must adjust man washroom door self closing device door close way must install back flow preventive device mop sink faucet'

In [35]:
#Create four vectorizations
#Unigram and Bigram & Count Vectorizer and TFIDF Vectorizer to compare
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import metrics

In [131]:
countvectorizer = CountVectorizer(max_features=1000, ngram_range=(1,2))
countvectorizer_matrix_train = countvectorizer.fit_transform(X_train)
countvectorizer_matrix_test = countvectorizer.transform(X_test)

In [132]:
countvectorizer.get_feature_names()

['accessible',
 'accordance',
 'accordance plumb',
 'accumulate',
 'accumulation',
 'across',
 'activity',
 'additional',
 'adequate',
 'adequately',
 'adequately trap',
 'air',
 'allergen',
 'allergen training',
 'along',
 'along wall',
 'also',
 'also clean',
 'also observe',
 'always',
 'approximately',
 'area',
 'area also',
 'area clean',
 'area floor',
 'area food',
 'area instruct',
 'area maintain',
 'area must',
 'area need',
 'area rear',
 'area remove',
 'area replace',
 'around',
 'around pipe',
 'article',
 'artificial',
 'artificial lighting',
 'attach',
 'available',
 'away',
 'away wall',
 'back',
 'back flow',
 'backflow',
 'bag',
 'bakery',
 'bar',
 'bar area',
 'base',
 'base corner',
 'baseboard',
 'basement',
 'basement area',
 'basement must',
 'basement storage',
 'bathroom',
 'beef',
 'behind',
 'behind bar',
 'behind cook',
 'behind equipment',
 'behind front',
 'bin',
 'black',
 'board',
 'board health',
 'book',
 'bottom',
 'bowl',
 'box',
 'boy',
 'break',
 

In [134]:
tfidfvectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1,2))
tfidfvectorizer_matrix_train = tfidfvectorizer.fit_transform(X_train)
tfidfvectorizer_matrix_test = tfidfvectorizer.transform(X_test)

In [135]:
tfidfvectorizer.get_feature_names()

['accessible',
 'accordance',
 'accordance plumb',
 'accumulate',
 'accumulation',
 'across',
 'activity',
 'additional',
 'adequate',
 'adequately',
 'adequately trap',
 'air',
 'allergen',
 'allergen training',
 'along',
 'along wall',
 'also',
 'also clean',
 'also observe',
 'always',
 'approximately',
 'area',
 'area also',
 'area clean',
 'area floor',
 'area food',
 'area instruct',
 'area maintain',
 'area must',
 'area need',
 'area rear',
 'area remove',
 'area replace',
 'around',
 'around pipe',
 'article',
 'artificial',
 'artificial lighting',
 'attach',
 'available',
 'away',
 'away wall',
 'back',
 'back flow',
 'backflow',
 'bag',
 'bakery',
 'bar',
 'bar area',
 'base',
 'base corner',
 'baseboard',
 'basement',
 'basement area',
 'basement must',
 'basement storage',
 'bathroom',
 'beef',
 'behind',
 'behind bar',
 'behind cook',
 'behind equipment',
 'behind front',
 'bin',
 'black',
 'board',
 'board health',
 'book',
 'bottom',
 'bowl',
 'box',
 'boy',
 'break',
 

#### Text Preprocessing Summary


1. Extract only comments from inspectors. This is to remove all the health codes from the texts and only focus on comments from the inspectors.
    - Split violations into lists for each inspection
    - Extract only the comment section
    - Concatenate the comments together
    
    
2. Pre-process comments for each inspection. This is to eliminate meaningless information such as punctuations as well as to allow combinations of word lemma.
    - Tokenize comments into words
    - Remove single characters
    - Remove numbers
    - Remove punctuations
    - Lowercase all words
    - Tag PoS for each words
    - Lemmatize words based on PoS
    
    
3. Vectorize text. This is performed after splitting train and test data to prevent data leakage.
    - Given that it is still uncertain which vectorizer would yield a better result, count vectorizer and TF-IDF vectorizer are used and compared. The data is vectorized
    - The number of Max Features is limited to 1000 to prevent the model from overfitting to train data.
    - N-Gram range is chosen to be between 1 and 2 given that the lengths of comments are relatively short. 

#### Model Selection & Evaluation

##### Naive Bayes

In [113]:
# instantiate a Multinomial Naive Bayes model
nb_tfidf = MultinomialNB()
nb_count = MultinomialNB()

In [114]:
# train and time the model using X_train_dtm
%time nb_count.fit(countvectorizer_matrix_train, y_train)
%time nb_tfidf.fit(tfidfvectorizer_matrix_train, y_train)

CPU times: user 287 ms, sys: 41 µs, total: 287 ms
Wall time: 284 ms
CPU times: user 298 ms, sys: 2 µs, total: 298 ms
Wall time: 296 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [115]:
# make class predictions for X_test_dtm
y_pred_class_train_nb_count = nb.predict(countvectorizer_matrix_train)
y_pred_class_test_nb_count = nb.predict(countvectorizer_matrix_test)
y_pred_class_train_nb_tfidf = nb.predict(tfidfvectorizer_matrix_train)
y_pred_class_test_nb_tfidf = nb.predict(tfidfvectorizer_matrix_test)

In [140]:
print(metrics.accuracy_score(y_train, y_pred_class_train_nb_count), metrics.accuracy_score(y_train, y_pred_class_train_nb_tfidf))

0.8459570058474585 0.8589842552156581


In [118]:
# calculate accuracy of class predictions
print(metrics.accuracy_score(y_test, y_pred_class_test_nb_count), metrics.accuracy_score(y_test, y_pred_class_test_nb_tfidf))

0.8457970442377439 0.8580345908070366


**Train & Test Data Result Comparison**

The accuracy scores between the train and test data of the models generated from both count-vectorized and TFIDF-vectorized inputs are very similar. Model overfitting is not observed in this case.

In [120]:
# calculate precision and recall
print(classification_report(y_test, y_pred_class_test_nb_count))
print(classification_report(y_test, y_pred_class_test_nb_tfidf))

                    precision    recall  f1-score   support

              Fail       0.79      0.70      0.74      9748
              Pass       0.93      0.93      0.93     24085
Pass w/ Conditions       0.65      0.74      0.69      6698

         micro avg       0.85      0.85      0.85     40531
         macro avg       0.79      0.79      0.79     40531
      weighted avg       0.85      0.85      0.85     40531

                    precision    recall  f1-score   support

              Fail       0.82      0.69      0.75      9748
              Pass       0.90      0.97      0.94     24085
Pass w/ Conditions       0.73      0.70      0.71      6698

         micro avg       0.86      0.86      0.86     40531
         macro avg       0.82      0.78      0.80     40531
      weighted avg       0.85      0.86      0.85     40531



In [121]:
# calculate the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_class_test_nb_count))
print(metrics.confusion_matrix(y_test, y_pred_class_test_nb_tfidf))

[[ 6865  1141  1742]
 [  742 22458   885]
 [ 1095   645  4958]]
[[ 6688  1632  1428]
 [  354 23431   300]
 [ 1132   908  4658]]


**Count Vectorizer and TFIDF Vectorizer comparison**

The Precisions, Recalls, and F-1 Scores calculated from the predictions from the models using count-vectorized and TF-IDF vectorized inputs are very comparable. The weighted average F-1 scores for both models are 0.85.

##### Support Vector Machine

In [1]:
from sklearn.svm import LinearSVC
lsvc_count = LinearSVC(class_weight='balanced')
lsvc_tfidf = LinearSVC(class_weight='balanced')

In [2]:
# train and time the model using X_train_dtm
%time lsvc_count.fit(countvectorizer_matrix_train, y_train)
%time lsvc_tfidf.fit(tfidfvectorizer_matrix_train, y_train)

NameError: name 'countvectorizer_matrix_train' is not defined

NameError: name 'tfidfvectorizer_matrix_train' is not defined

In [143]:
# make class predictions for X_test_dtm
y_pred_class_train_lsvc_count = lsvc_count.predict(countvectorizer_matrix_train)
y_pred_class_test_lsvc_count = lsvc_count.predict(countvectorizer_matrix_test)
y_pred_class_train_lsvc_tfidf = lsvc_tfidf.predict(tfidfvectorizer_matrix_train)
y_pred_class_test_lsvc_tfidf = lsvc_tfidf.predict(tfidfvectorizer_matrix_test)

In [144]:
print(metrics.accuracy_score(y_train, y_pred_class_train_lsvc_count), metrics.accuracy_score(y_train, y_pred_class_train_lsvc_tfidf))

0.9303169047593871 0.92985164585338


In [145]:
# calculate accuracy of class predictions
print(metrics.accuracy_score(y_test, y_pred_class_test_lsvc_count), metrics.accuracy_score(y_test, y_pred_class_test_lsvc_tfidf))

0.9249463373713948 0.9264760306925564


**Train & Test Data Result Comparison**

The accuracy scores between the train and test data of the models generated from both count-vectorized and TFIDF-vectorized inputs are very similar. Model overfitting is not observed in this case.

In [146]:
# calculate precision and recall
print(classification_report(y_test, y_pred_class_test_lsvc_count))
print(classification_report(y_test, y_pred_class_test_lsvc_tfidf))

                    precision    recall  f1-score   support

              Fail       0.88      0.84      0.86      9748
              Pass       0.96      0.99      0.98     24085
Pass w/ Conditions       0.85      0.80      0.83      6698

         micro avg       0.92      0.92      0.92     40531
         macro avg       0.90      0.88      0.89     40531
      weighted avg       0.92      0.92      0.92     40531

                    precision    recall  f1-score   support

              Fail       0.90      0.83      0.86      9748
              Pass       0.96      0.99      0.98     24085
Pass w/ Conditions       0.83      0.84      0.83      6698

         micro avg       0.93      0.93      0.93     40531
         macro avg       0.90      0.89      0.89     40531
      weighted avg       0.93      0.93      0.93     40531



In [147]:
# calculate the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_class_test_lsvc_count))
print(metrics.confusion_matrix(y_test, y_pred_class_test_lsvc_tfidf))

[[ 8186   669   893]
 [  109 23918    58]
 [ 1001   312  5385]]
[[ 8051   632  1065]
 [  119 23890    76]
 [  801   287  5610]]


**Count Vectorizer and TFIDF Vectorizer Comparison**

Similar to the NB Classifier models, the Precisions, Recalls, and F-1 Scores calculated from the predictions from the models using count-vectorized and TF-IDF vectorized inputs are very comparable. However, Linear Support Vector Machine Classifier performs significantly better, yielding the weighted average F-1 scores for both models of 0.93.